In [ ]:
import json
import requests
import time
from urllib.parse import unquote


class QZonePhotoCrawler:
    def __init__(self, albumid=None):
        self.base_url = "https://h5.qzone.qq.com/webapp/json/mqzone_photo/getPhotoList2"
        self.params = {
            "qzonetoken": "6b3181c0885b3ee2c5228d2d8860020ac60fb0a7df3150189d6b87a4d6a14a9c9e0e07c0becae1f0a93adc5370a8710e",
            "g_tk": "1692888408",
            "uin": "405720329",
            "albumid": albumid,
            "ps": "0",  # 初始值为0，后续会递增
            "pn": "200",  # 固定每次请求200张
            "password": "",
            "password_cleartext": "0",
            "swidth": "1278.000038087368",
            "sheight": "1604.000047802925",
        }
        self.headers = {
            "accept": "application/json",
            "accept-language": "zh-CN,zh;q=0.9,en;q=0.8",
            "priority": "u=1, i",
            "referer": "https://h5.qzone.qq.com/mqzone/profile?hostuin=405720329&no_topbar=1&srctype=10&stat=&g_f=2000000209",
            "sec-ch-ua": '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
            "sec-ch-ua-mobile": "?1",
            "sec-ch-ua-platform": '"Android"',
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Mobile Safari/537.36",
            "x-requested-with": "XMLHttpRequest",
        }
        self.cookies = {
            "ptcz": "4521759dd4d7efe5ecfe4870304c908e5d76f1fbc259019de3d9dc877447887f",
            "pgv_pvid": "4255283212",
            "qq_domain_video_guid_verify": "148b485d52fd84d0",
            "_qimei_q36": "",
            "_qimei_h38": "19c526467a14828b782dda690200000e618413",
            "o_cookie": "405720329",
            "iip": "0",
            "_t_qbtool_uid": "aaaa5uhl2ftc71u3n6hisg2rwmgy88cb",
            "_ga": "GA1.1.810392572.1720143751",
            "_ga_TPFW0KPXC1": "GS1.1.1720143750.1.1.1720143802.0.0.0",
            "pac_uid": "0_jRkAMrRkjDD35",
            "suid": "user_0_jRkAMrRkjDD35",
            "_qimei_fingerprint": "8c6d3b400d828b5aa48094d943725a13",
            "fqm_pvqid": "871f0d36-a9e9-4bce-baf0-f8a8ed2da36c",
            "RK": "VGMEVx9LUG",
            "Loading": "Yes",
            "QZ_FE_WEBP_SUPPORT": "1",
            "cpu_performance_v8": "0",
            "uin": "o0405720329",
            "skey": "@HCpdgU64n",
            "p_uin": "o0405720329",
            "pt4_token": "joqfMcHH9uNVkggGiQ0zU8bRXwlcjTJwm0V4hrJUDEg_",
            "p_skey": "DnRx9urgCw9IuY18P9H7sdpZZrU3lm5TurWOyd1BfEw_",
            "tgw_l7_route": "ecd4bcf87e08055cd90646261ab6e142",
            "pgv_info": "ssid=s6172181052",
            "_qz_referrer": "user.qzone.qq.com",
        }
        self.all_photos = []
        self.total_count = 0

    def fetch_photos(self):
        """Fetch photos in batches of 200"""
        while True:
            print(
                f"Requesting photos {int(self.params['ps'])}-{int(self.params['ps']) + 200}..."
            )

            try:
                response = requests.get(
                    self.base_url,
                    params=self.params,
                    headers=self.headers,
                    cookies=self.cookies,
                    timeout=30,
                )
                data = response.json()
            except Exception as e:
                print(f"Request failed: {e}")
                break

            # 首次请求获取总数
            if self.total_count == 0:
                self.total_count = data.get("data", {}).get("total_count", 0)
                self.name = data.get("data", {})["album"]["name"]
                if self.total_count == 0:
                    print("No photos found in album")
                    break
                print(f"Total photos to fetch: {self.total_count}")

            # 处理照片数据
            photos_data = data.get("data", {}).get("photos", {})
            for photo_list in photos_data.values():
                if isinstance(photo_list, list):
                    self.all_photos.extend(photo_list)

            # 更新已获取数量
            self.params["ps"] = str(len(self.all_photos))
            print(f"Progress: {len(self.all_photos)}/{self.total_count}")

            # 检查是否完成
            if len(self.all_photos) >= self.total_count:
                break

            time.sleep(2)  # 请求间隔

    def save_results(self):
        """Save combined results"""
        if not self.all_photos:
            print("No photos to save")
            return

        result = {
            "albumid": self.params["albumid"],
            "name": self.name,
            "total_count": self.total_count,
            "fetched_count": len(self.all_photos),
            "photos": self.all_photos,
        }

        filename = f"gpx_json_json\\{self.name}_combined_photos.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(result, f, ensure_ascii=False, indent=4)
        print(f"Saved {len(self.all_photos)} photos to {filename}")

    def run(self):
        self.fetch_photos()
        self.save_results()


if __name__ == "__main__":
    id_list = (
        [
            "V13Wf2g01Hmcwj",
            "V13Wf2g0084wFG",
            "V13Wf2g04en0tg",
            "V13Wf2g01gVklS",
            "V13Wf2g01zErkC",
            "V13Wf2g00eexCP",
            "V13Wf2g00IiH7b",
            "V13Wf2g012I8Me",
            "V13Wf2g02EBkS9",
            "V13Wf2g01GX2si",
        ]
        + [
            "V13Wf2g02R720j",
            "V13Wf2g00hhdov",
            "V13Wf2g04dMQTr",
            "V13Wf2g01JXw0G",
            "V13Wf2g01ZwCNH",
            "V13Wf2g03veG1C",
            "V13Wf2g03SCSzg",
            "V13Wf2g04DGqak",
            "V13Wf2g01fC80n",
            "V13Wf2g00YAbbm",
        ]
        + [
            "V513poEq2wufYD0XFeHQ0aZpQj0WDzox",
            "V513poEq2wufYD0XFeHQ0aZpQj0jDaoY",
            "V513poEq2wufYD0XFeHQ0aZpQj4aN6sN",
            "V513poEq2wufYD0XFeHQ0aZpQj1yNFE9",
            "V513poEq2wufYD0XFeHQ0aZpQj3HMCxs",
            "V13Wf2g02ekRVZ",
            "V513poEq2wufYD0XFeHQ0aZpQj14xHeQ",
            "V513poEq2wufYD0XFeHQ0aZpQj00o2yE",
            "V513poEq2wufYD0XFeHQ0aZpQj3GH6uf",
            "V13Wf2g02iDrbN",
        ]
        + [
            "V13Wf2g01zErkC",
            "V13Wf2g01Hmcwj",
            "V13Wf2g0084wFG",
            "V13Wf2g02EBkS9",
            "V13Wf2g00IiH7b",
            "V13Wf2g04en0tg",
            "V13Wf2g00eexCP",
            "V13Wf2g01GX2si",
            "V13Wf2g01gVklS",
            "V13Wf2g012I8Me",
        ]
        + [
            "V13Wf2g01ZwCNH",
            "V13Wf2g00YAbbm",
            "V13Wf2g03veG1C",
            "V13Wf2g00hhdov",
            "V13Wf2g04DGqak",
            "V13Wf2g04dMQTr",
            "V13Wf2g01JXw0G",
            "V13Wf2g02R720j",
            "V13Wf2g01fC80n",
            "V13Wf2g03SCSzg",
        ]
        + [
            "V13Wf2g00n4qB9",
            "V13Wf2g03NnwYy",
            "V13Wf2g042MsgX",
            "V13Wf2g01YGBK0",
            "V13Wf2g03B3MNn",
            "V13Wf2g04YKzIy",
            "V13Wf2g02VuoJy",
            "V13Wf2g04DWuTS",
            "V13Wf2g03ue2Kl",
            "V13Wf2g032YL9U",
        ]
    )
    print(f"Total albums to process: {len(id_list)}")
    for albumid in id_list:
        print(f"Processing album ID: {albumid}")
        crawler = QZonePhotoCrawler(albumid=albumid)
        crawler.run()
        print(f"Finished processing album ID: {albumid}\n")
        time.sleep(2)

In [21]:
import re

a = re.findall(r'"albumid": "(.+?)",', open("1.json", "r", encoding="utf-8").read())
a = list(set(a))
a

['V13Wf2g00n4qB9',
 'V13Wf2g03NnwYy',
 'V13Wf2g042MsgX',
 'V13Wf2g01YGBK0',
 'V13Wf2g03B3MNn',
 'V13Wf2g04YKzIy',
 'V13Wf2g02VuoJy',
 'V13Wf2g04DWuTS',
 'V13Wf2g03ue2Kl',
 'V13Wf2g032YL9U']

In [ ]:
# 以及这个还是不能很好处理坐标为 0 的情况，我不知道是啥情况，但反正也不太多，手动处理吧
import os
import json
import random
import datetime
import xml.etree.ElementTree as ET
import gpxpy

def timestamp_to_local_filename(ts):
    dt = datetime.datetime.fromtimestamp(ts, datetime.timezone(datetime.timedelta(hours=8)))
    return dt.strftime("%Y-%m-%d")

def write_gpx(date_key, points, output_dir):
    gpx = gpxpy.gpx.GPX()
    gpx_track = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(gpx_track)
    gpx_segment = gpxpy.gpx.GPXTrackSegment()
    gpx_track.segments.append(gpx_segment)

    points.sort()  # 按 UTC 时间排序

    for dt, lat, lon, alt in points:
        if lat < 1 or lon < 1:
            a = 1
        pt = gpxpy.gpx.GPXTrackPoint(lat, lon, elevation=alt, time=dt)
        gpx_segment.points.append(pt)

    output_path = os.path.join(output_dir, f"{date_key}.gpx")
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(gpx.to_xml())

def add_fake_point(point):
    dt, lat, lon, alt = point
    new_dt = dt + datetime.timedelta(seconds=60)
    lat_offset = 0.00050
    lon_offset = 0.00050
    new_lat = lat + lat_offset
    new_lon = lon + lon_offset
    if alt is not None:
        alt_offset = random.uniform(-1, 1)
        new_alt = alt + alt_offset
    else:
        new_alt = None
    return (new_dt, new_lat, new_lon, new_alt)

# 主处理逻辑
def process_json_folder(input_dir, output_dir):
    all_points_by_day = {}

    for filename in os.listdir(input_dir):
        if filename.endswith('.json'):
            filepath = os.path.join(input_dir, filename)
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)

            photos = data.get('photos', [])
            for photo in photos:
                shoottime = photo.get('shoottime', 0)
                if shoottime == 0:
                    shoottime = photo.get('uUploadTime', 0)
                if shoottime == 0:
                    continue  # 没有时间就跳过

                if shoottime == 1557219617:
                    a = 1
                geo = photo.get('geo', {})
                poi_x = geo.get('poi_x')
                poi_y = geo.get('poi_y')
                if poi_x == '' or poi_y == '' or abs(float(poi_x) - 0.0) < 1 or abs(float(poi_y) - 0.0) < 1:
                    continue  # 没有地理信息就跳过

                lon = float(poi_x)
                lat = float(poi_y)
                
                if lon < 1 or lat < 1:
                    continue

                
                tz = datetime.timezone(datetime.timedelta(hours=8))
                local_dt = datetime.datetime.fromtimestamp(shoottime, tz)
                utc_dt = local_dt.astimezone(datetime.timezone.utc)
                date_key = local_dt.date()  # 只取年月日作为key

                if date_key not in all_points_by_day:
                    all_points_by_day[date_key] = []

                all_points_by_day[date_key].append((utc_dt, lat, lon, None))  # 暂时不处理海拔（alt）

    # 生成每天的GPX
    for date_key, points in all_points_by_day.items():
        if len(points) == 1:
            # 只有一个点时，加一个扰动点
            fake_point = add_fake_point(points[0])
            points.append(fake_point)

        # 按时间排序
        points.sort(key=lambda x: x[0])

        start_ts = int(points[0][0].timestamp())
        write_gpx(timestamp_to_local_filename(start_ts), points, output_dir)

if __name__ == "__main__":
    input_dir = r"D:\image\yeah\gpx_json_json"
    output_dir = r"D:\image\yeah\gpx_output"
    process_json_folder(input_dir, output_dir)
